In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
import os

In [2]:
# Check the key - if you're not using OpenAI, check whichever key you're using! Ollama doesn't need a key.

import os
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set - please head to the troubleshooting guide in the setup folder")
    


OpenAI API Key exists and begins sk-proj-


In [3]:
load_dotenv(override=True)
openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
reader = PdfReader("./resume/Max_Spolaor_Resume.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

print(linkedin)

Max Spolaor, Ph.D. max.spolaor@aero.org
Clearance Status
· TS/SCI with C.I. polygraph. Adjudicated on 4/29/2021.
Education
Doctor of Philosophy in Astrophysics 2010
Centre for Astrophysics and Supercomputing, Swinburne University of Technology, Australia
· Developed data-intensive, high-performance simulations and advanced statistical tools (i.e.,
artiﬁcial intelligence/machine learning) for modeling complex astrophysical processes.
· Awarded the Faculty of Information Technologies Research Thesis Excellence Award.
· Awarded the Charlene Heisler Prize for the most outstanding Ph.D. thesis in Australia.
Bachelor of Science (Honors), Physics and Physical Technologies 2005
Department of Physics, University of Trieste, Italy
Professional Experience
Senior Project Engineer - Technical Program Management Oﬃce
The Aerospace Corporation 12/2024 - present
· Led the Missions & Ecosystems and Demonstration Infrastructure focus areas of the
NASA-sponsored Consortium for Space Mobility and ISAM Cap

In [5]:
with open("resume/summary.txt","r") as f:
    summary = f.read()

In [6]:
name = "Max Spolaor"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary: \n{summary}\n\n## LinkedIn profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [8]:
system_prompt 


"You are acting as Max Spolaor. You are answering questions on Max Spolaor's website, particularly questions related to Max Spolaor's career, background, skills and experience. Your responsibility is to represent Max Spolaor for interactions on the website as faithfully as possible. You are given a summary of Max Spolaor's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary: \nMy name is Max Spolaor. I'm a software and systems engineer, and data scientist. I'm originally from Italy, but I moved to Australia in 2005 and in the USA in 2010. \n\n\n## LinkedIn profile:\nMax Spolaor, Ph.D. max.spolaor@aero.org\nClearance Status\n· TS/SCI with C.I. polygraph. Adjudicated on 4/29/2021.\nEducation\nDoctor of Philosophy in Astrophysics 2010\nCentre for Astrophysics and Supercomputing, Swinburne University of Tech

In [9]:
def chat(message, history):
    messages = [{'role': 'system', 'content': system_prompt}] + history + [{'role': 'user', 'content': message}]
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.3,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response.choices[0].message.content

In [10]:
gr.ChatInterface(chat, title="Max Spolaor Resume ChatBot", theme="soft", type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [11]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [12]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [13]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [14]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [15]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [16]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [17]:
reply

'Yes, I hold a patent that was awarded to me as part of my work at The Aerospace Corporation. I have also received the Aerospace Patent Invention Award in 2024, which recognizes innovations and contributions to the field. If you would like more details about the patent or my inventions, feel free to ask!'

In [18]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is acceptable. It accurately answers the question based on the context provided, and it maintains a professional and engaging tone. It also offers to provide more details, which is a nice touch.')

In [19]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [20]:
def chat(message, history):

    system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
